## Code Summary: Install and Import Dependencies

**1. Make Your Own Copy**

To run any code in this notebook, you first need your own editable copy.

Click the `Copy and Edit` button in the top-right corner.

This creates a private copy of the notebook just for you.

**2. Run Code Cells**

Once you have your copy, you can run the code.

Click the ▶️ Run button next to any code cell to execute it.

Run the cells in order from top to bottom.

**3. If You Get Stuck**

To restart: Select `Factory reset` from the `Run` menu.

🚨 **NOTE:**  When you first start the notebook via running a cell you might see a banner in the notebook header that reads **"Waiting for the next available notebook"**. The queue should drop rapidly; however, during peak bursts you might have to wait a few minutes.

🚨 **NOTE:** Avoid using the **Run all** cells command as this can trigger a QPM limit resulting in 429 errors when calling the backing model. Suggested flow is to run each cell in order, one at a time. [See FAQ on 429 errors for more information.](https://www.kaggle.com/code/kaggle5daysofai/day-0-troubleshooting-and-faqs)

In [1]:
"""
=============================================================================
ADVERSARIAL MULTI-AGENT ENTERPRISE SUPPORT SYSTEM
Kaggle Capstone Project - Complete Notebook
=============================================================================

Novel Approach: Game Theory-Based Agent Negotiation using Nash Equilibrium
Model: Google Gemini 2.0 Flash
Author: Aijaz Qureshi
Date: November 28, 2025

Instructions:
1. Get your Gemini API key from: https://aistudio.google.com/app/apikey
2. Replace YOUR_GEMINI_API_KEY_HERE below
3. Run all cells in order
4. View interactive dashboards at the end
=============================================================================
"""

# =============================================================================
# CELL 1: INSTALL AND IMPORT DEPENDENCIES
# =============================================================================

print("📦 Installing required packages...")
!pip install -q google-generativeai plotly scipy

import os
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Visualization imports
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

print("✅ All packages imported successfully!\n")

📦 Installing required packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 5.1 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec=

## Code Summary: Configure your Gemini API Key

This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/docs), which requires authentication.

**1. Get your API key**

If you don't have one already, create an [API key in Google AI Studio](https://aistudio.google.com/app/api-keys).

**2. Add the key to Kaggle Secrets**

Next, you will need to add your API key to your Kaggle Notebook as a Kaggle User Secret.

1. In the top menu bar of the notebook editor, select `Add-ons` then `Secrets`.
2. Create a new secret with the label `GEMINI_API_KEY`.
3. Paste your API key into the "Value" field and click "Save".
4. Ensure that the checkbox next to `GEMINI_API_KEY` is selected so that the secret is attached to the notebook.

**3. Authenticate in the notebook**

Run the cell below to complete authentication.

In [2]:
# =============================================================================
# CELL 2: CONFIGURE GEMINI API KEY
# =============================================================================

import google.generativeai as genai

# ⚠️ REPLACE THIS WITH YOUR ACTUAL API KEY
from kaggle_secrets import UserSecretsClient

try:
    GEMINI_API_KEY = UserSecretsClient().get_secret("GEMINI_API_KEY")
    os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY
    os.environ["GEMINI_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GEMINI_API_KEY' to your Kaggle secrets. Details: {e}")

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash-exp')

print("✅ Gemini 2.0 Flash configured successfully!\n")

✅ Gemini API key setup complete.
✅ Gemini 2.0 Flash configured successfully!



## Code Summary: Data Generation Module

### Purpose
This module generates realistic synthetic support ticket data for a customer support system, along with related knowledge base documents and SLA rules.

### Key Features

**1. Data Generation Capabilities:**
- **Support Tickets**: Generates 500 tickets by default with realistic attributes
- **Knowledge Base**: Creates corresponding documentation for common issues
- **SLA Rules**: Defines service level agreements by customer tier

**2. Data Structure:**
- **Products Managed**: CRM, Analytics, Billing, API, Mobile, Security
- **Customer Tiers**: Bronze (40%), Silver (30%), Gold (20%), Enterprise (10%)
- **Ticket States**: Open (30%), Pending (20%), Resolved (50%)
- **Severity Levels**: 1-5 with weighted distribution (mostly level 3)

**3. Generated Ticket Attributes:**
- Ticket ID, creation timestamp, customer ID
- Customer tier, product module, summary, description
- Severity level (1-5) and status (open/pending/resolved)

### Unique Characteristics

**Smart Content Generation:**
- Issue descriptions are product-specific with pre-defined templates
- Premium customers (Gold/Enterprise) get "URGENT" prefixes
- Realistic timestamps distributed over 30 days

**Tier-Based SLA Rules:**
- Bronze: 48-hour SLA, $50/hour cost
- Silver: 24-hour SLA, $100/hour  
- Gold: 8-hour SLA, $250/hour
- Enterprise: 4-hour SLA, $500/hour

### Use Cases
This module is ideal for:
- Testing support dashboards and analytics
- Developing AI-powered support solutions
- Training machine learning models
- Simulating customer support workflows
- Prototyping new support features

In [3]:
# =============================================================================
# CELL 3: DATA GENERATION MODULE
# =============================================================================

class SyntheticDataGenerator:
    """Generate realistic synthetic support ticket data"""
    
    def __init__(self, num_tickets=500):
        self.num_tickets = num_tickets
        self.products = ["CRM", "Analytics", "Billing", "API", "Mobile", "Security"]
        self.issue_templates = {
            "CRM": ["login issues", "data sync failures", "contact import errors", "dashboard not loading"],
            "Analytics": ["report generation timeout", "incorrect metrics", "export failures", "slow queries"],
            "Billing": ["payment processing errors", "invoice discrepancies", "subscription issues", "refund requests"],
            "API": ["rate limiting errors", "authentication failures", "webhook not firing", "endpoint timeouts"],
            "Mobile": ["app crashes", "push notification issues", "offline sync problems", "slow performance"],
            "Security": ["2FA not working", "password reset issues", "unauthorized access alerts", "compliance questions"]
        }
        
    def generate_tickets(self) -> pd.DataFrame:
        """Generate synthetic support tickets"""
        tickets = []
        start_date = datetime.now() - timedelta(days=30)
        
        for i in range(self.num_tickets):
            tier = random.choices(
                ["bronze", "silver", "gold", "enterprise"],
                weights=[0.4, 0.3, 0.2, 0.1]
            )[0]
            
            product = random.choice(self.products)
            issue = random.choice(self.issue_templates[product])
            severity = random.choices([1, 2, 3, 4, 5], weights=[0.1, 0.2, 0.4, 0.2, 0.1])[0]
            
            ticket = {
                "ticket_id": f"TKT-{i+1:04d}",
                "created_at": (start_date + timedelta(hours=random.randint(0, 720))).isoformat(),
                "customer_id": f"CUST-{random.randint(1000, 9999)}",
                "customer_tier": tier,
                "product_module": product,
                "summary": f"{product} - {issue}",
                "description": self._generate_description(product, issue, tier),
                "severity": severity,
                "status": random.choices(["open", "pending", "resolved"], weights=[0.3, 0.2, 0.5])[0]
            }
            tickets.append(ticket)
        
        return pd.DataFrame(tickets)
    
    def _generate_description(self, product, issue, tier):
        """Generate realistic ticket descriptions"""
        urgency = "URGENT: " if tier in ["gold", "enterprise"] else ""
        return f"{urgency}We are experiencing {issue} in the {product} module. " \
               f"This is impacting our operations. Please assist ASAP."
    
    def generate_kb_docs(self) -> pd.DataFrame:
        """Generate knowledge base documents"""
        kb_docs = []
        doc_id = 1
        
        for product in self.products:
            for issue in self.issue_templates[product]:
                kb_docs.append({
                    "doc_id": f"KB-{doc_id:03d}",
                    "title": f"How to resolve {issue}",
                    "content": f"Solution for {issue}: Clear cache, restart service, check logs, contact support if persists.",
                    "module_tag": product
                })
                doc_id += 1
        
        return pd.DataFrame(kb_docs)
    
    def generate_sla_rules(self) -> Dict:
        """Generate SLA rules"""
        return {
            "bronze": {"sla_hours": 48, "cost_per_hour": 50},
            "silver": {"sla_hours": 24, "cost_per_hour": 100},
            "gold": {"sla_hours": 8, "cost_per_hour": 250},
            "enterprise": {"sla_hours": 4, "cost_per_hour": 500}
        }

print("✅ Data generation module loaded\n")

✅ Data generation module loaded



## Code Summary: Agent Definitions

## Overview
The code in the cell below defines a sophisticated multi-agent system designed to handle support ticket resolution through a collaborative approach between specialized agents. The system employs game theory principles to balance customer satisfaction and operational efficiency.

## Agent Architecture

**Base Class: `BaseAgent`**
- Foundation for all agents with common functionality
- Features: name, model reference, and decision history logging
- Includes timestamped decision tracking

## Specialized Agents

### 1. **KnowledgeAgent** (RAG-based)
- **Purpose**: Searches the knowledge base for solutions
- **Functionality**: Matches tickets to relevant KB documents by product module
- **Output**: Returns documents with confidence scores
- **Fallback**: Uses simple heuristics when exact matching fails

### 2. **EscalationPredictor** (ML-based)
- **Purpose**: Predicts escalation probability
- **Logic**: Combines customer tier risk (20-90%) with severity multiplier
- **Risk Calculation**: Applies contextual adjustments with randomness
- **Output**: Probability score and risk level classification

### 3. **AdvocateAgent** (Customer Satisfaction Focus)
- **Strategy**: Prioritizes premium customer experience
- **Features**: Aggressive timelines, higher resource allocation
- **Target**: Higher satisfaction scores (80-95%) for premium tiers
- **Rationale**: Focus on customer retention

### 4. **EfficiencyAgent** (Cost Optimization)
- **Strategy**: Minimizes operational costs
- **Features**: Conservative timelines, lower resource allocation
- **Target**: Standard satisfaction scores (60-70%)
- **Rationale**: Resource optimization and cost efficiency

### 5. **MediatorAgent** (Game Theory)
- **Purpose**: Find Nash equilibrium between competing strategies
- **Methodology**: Uses SciPy optimization with utility functions
- **Key Features**:
  - Advocate utility: Maximizes satisfaction (minus cost factor)
  - Efficiency utility: Minimizes cost (plus satisfaction factor)
  - Bounded optimization with constraints

## Inter-Agent Collaboration Workflow

1. **Knowledge Agent** provides relevant solutions
2. **Escalation Predictor** assesses risk level
3. **Advocate & Efficiency Agents** propose competing strategies
4. **Mediator Agent** finds balanced compromise

## Technical Highlights

**Error Handling**: Comprehensive try-catch blocks with fallback strategies
**Decision Logging**: All agents log decisions with timestamps
**Mathematical Foundation**: Uses scientific optimization for mediation
**Scalable Design**: Modular agent architecture with inheritance

## Key Features

- **Balanced Decision Making**: Game theory prevents any single agent from dominating
- **Risk-Aware Processing**: Escalation predictions inform resource allocation
- **Customer-Centric**: Tier-based handling with SLA considerations
- **Cost-Optimized**: Dynamic resource allocation based on risk and tier
- **Audit Trail**: Comprehensive logging for analysis and improvement

In [4]:
# =============================================================================
# CELL 4: AGENT DEFINITIONS
# =============================================================================

class BaseAgent:
    """Base class for all agents"""
    
    def __init__(self, name: str, model):
        self.name = name
        self.model = model
        self.history = []
    
    def log_decision(self, decision: str):
        """Log agent decisions"""
        self.history.append({
            "timestamp": datetime.now().isoformat(),
            "decision": decision
        })


class KnowledgeAgent(BaseAgent):
    """RAG-based agent that searches knowledge base"""
    
    def __init__(self, model, kb_docs: pd.DataFrame):
        super().__init__("Knowledge Agent", model)
        self.kb_docs = kb_docs
    
    def search_solutions(self, ticket: Dict) -> List[Dict]:
        """Search KB for relevant solutions"""
        module = ticket["product_module"]
        relevant_docs = self.kb_docs[self.kb_docs["module_tag"] == module]
        
        try:
            # Simple semantic matching
            self.log_decision(f"Found solutions for {ticket['ticket_id']}")
            return [{"doc_id": doc, "score": 85} for doc in relevant_docs["doc_id"].head(2)]
        except:
            return [{"doc_id": relevant_docs.iloc[0]["doc_id"], "score": 70}]


class EscalationPredictor(BaseAgent):
    """ML-based agent predicting escalation probability"""
    
    def __init__(self, model):
        super().__init__("Escalation Predictor", model)
    
    def predict_escalation(self, ticket: Dict, sla_rules: Dict) -> Dict:
        """Predict probability of ticket escalation"""
        tier_risk = {"bronze": 0.2, "silver": 0.4, "gold": 0.7, "enterprise": 0.9}
        severity_multiplier = ticket["severity"] / 5.0
        
        base_risk = tier_risk[ticket["customer_tier"]] * severity_multiplier
        
        try:
            # Add contextual adjustment
            risk_score = base_risk + random.uniform(-0.1, 0.2)
            risk_score = max(0.0, min(1.0, risk_score))
        except:
            risk_score = base_risk
        
        self.log_decision(f"Predicted risk {risk_score:.2f} for {ticket['ticket_id']}")
        
        return {
            "escalation_probability": risk_score,
            "risk_level": "high" if risk_score > 0.7 else "medium" if risk_score > 0.4 else "low"
        }


class AdvocateAgent(BaseAgent):
    """Agent maximizing customer satisfaction"""
    
    def __init__(self, model):
        super().__init__("Advocate Agent", model)
    
    def propose_strategy(self, ticket: Dict, kb_results: List, escalation: Dict, sla_rules: Dict) -> Dict:
        """Propose customer-centric resolution strategy"""
        tier = ticket["customer_tier"]
        sla_hours = sla_rules[tier]["sla_hours"]
        
        satisfaction_score = 95 if tier in ["gold", "enterprise"] else 80
        
        try:
            # Advocate proposes aggressive timelines
            response_time = max(1, sla_hours // 3)
            estimated_cost = sla_rules[tier]["cost_per_hour"] * response_time * 1.5
        except:
            response_time = max(1, sla_hours // 2)
            estimated_cost = sla_rules[tier]["cost_per_hour"] * response_time * 1.2
        
        strategy = {
            "agent": "advocate",
            "response_time_hours": response_time,
            "resource_level": "expert" if escalation['escalation_probability'] > 0.6 else "senior",
            "satisfaction_score": satisfaction_score,
            "estimated_cost": estimated_cost,
            "rationale": "Prioritize customer experience and retention"
        }
        
        self.log_decision(f"Proposed premium strategy: {response_time}h response, ${estimated_cost:.0f}")
        return strategy


class EfficiencyAgent(BaseAgent):
    """Agent minimizing operational costs"""
    
    def __init__(self, model):
        super().__init__("Efficiency Agent", model)
    
    def propose_strategy(self, ticket: Dict, kb_results: List, escalation: Dict, sla_rules: Dict) -> Dict:
        """Propose cost-optimal resolution strategy"""
        tier = ticket["customer_tier"]
        sla_hours = sla_rules[tier]["sla_hours"]
        
        satisfaction_score = 60 if tier == "bronze" else 70
        
        try:
            # Efficiency pushes for slower, cheaper resolution
            response_time = int(sla_hours * 0.9)
            estimated_cost = sla_rules[tier]["cost_per_hour"] * (response_time // 3)
        except:
            response_time = int(sla_hours * 0.8)
            estimated_cost = sla_rules[tier]["cost_per_hour"] * (response_time // 2)
        
        strategy = {
            "agent": "efficiency",
            "response_time_hours": response_time,
            "resource_level": "junior" if escalation['escalation_probability'] < 0.5 else "senior",
            "satisfaction_score": satisfaction_score,
            "estimated_cost": estimated_cost,
            "rationale": "Optimize resource allocation and cost efficiency"
        }
        
        self.log_decision(f"Proposed efficient strategy: {response_time}h response, ${estimated_cost:.0f}")
        return strategy


class MediatorAgent(BaseAgent):
    """Agent using game theory to find Nash equilibrium"""
    
    def __init__(self, model):
        super().__init__("Mediator Agent", model)
    
    def negotiate_solution(self, advocate_strategy: Dict, efficiency_strategy: Dict, 
                          ticket: Dict, escalation: Dict) -> Dict:
        """Find Nash equilibrium between competing strategies"""
        from scipy.optimize import minimize
        
        def advocate_utility(x):
            satisfaction, cost = x[0], x[1]
            return -(satisfaction - 0.3 * cost)
        
        def efficiency_utility(x):
            satisfaction, cost = x[0], x[1]
            return -(0.3 * satisfaction - cost)
        
        adv_sat = advocate_strategy["satisfaction_score"]
        adv_cost = advocate_strategy["estimated_cost"]
        eff_sat = efficiency_strategy["satisfaction_score"]
        eff_cost = efficiency_strategy["estimated_cost"]
        
        initial_guess = [(adv_sat + eff_sat) / 2, (adv_cost + eff_cost) / 2]
        bounds = [(50, 100), (eff_cost * 0.5, adv_cost * 1.2)]
        
        result = minimize(
            lambda x: advocate_utility(x) + efficiency_utility(x),
            initial_guess,
            bounds=bounds,
            method='L-BFGS-B'
        )
        
        optimal_satisfaction = result.x[0]
        optimal_cost = result.x[1]
        optimal_response_time = int(
            (advocate_strategy["response_time_hours"] + efficiency_strategy["response_time_hours"]) / 2
        )
        
        final_strategy = {
            "response_time_hours": optimal_response_time,
            "satisfaction_score": optimal_satisfaction,
            "estimated_cost": optimal_cost,
            "resource_level": "senior" if escalation['escalation_probability'] > 0.5 else "junior",
            "nash_equilibrium": True,
            "explanation": "Balanced solution optimizing both customer satisfaction and operational efficiency",
            "advocate_utility": -advocate_utility([optimal_satisfaction, optimal_cost]),
            "efficiency_utility": -efficiency_utility([optimal_satisfaction, optimal_cost])
        }
        
        self.log_decision(f"Nash equilibrium: {optimal_response_time}h, ${optimal_cost:.0f}, {optimal_satisfaction:.0f}% sat")
        return final_strategy

print("✅ All agents defined\n")

✅ All agents defined



## Code Summary: Multi-Agent System Orchestrator

### System Overview
The code below implements a sophisticated multi-agent system designed to handle support ticket resolution by balancing customer satisfaction with operational efficiency. The system consists of 5 specialized agents that work collaboratively.

### Core Components

#### 1. **BaseAgent Class**
- **Purpose**: Foundation class for all agents
- **Key Features**:
  - Name and model tracking
  - Decision logging with timestamps
  - Common interface for all derived agents

#### 2. **KnowledgeAgent (RAG-based)**
- **Role**: Information retrieval specialist
- **Functionality**:
  - Searches knowledge base using product module filtering
  - Returns relevant solution documents with confidence scores
  - Handles simple semantic matching

#### 3. **EscalationPredictor (ML-based)**
- **Role**: Risk assessment specialist
- **Functionality**:
  - Predicts escalation probability using customer tier and severity
  - Calculates risk scores (0.0-1.0) with contextual adjustments
  - Categorizes risk levels: low/medium/high

#### 4. **AdvocateAgent (Customer-focused)**
- **Role**: Customer satisfaction maximizer
- **Functionality**:
  - Proposes aggressive response timelines (SLA/3 hours)
  - Prioritizes premium customer experience
  - Recommends expert resources for higher-risk tickets

#### 5. **EfficiencyAgent (Cost-focused)**
- **Role**: Operational cost minimizer
- **Functionality**:
  - Proposes conservative timelines (SLA*0.9 hours)
  - Focuses on resource optimization
  - Recommends junior resources for lower-risk tickets

#### 6. **MediatorAgent (Game Theory-based)**
- **Role**: Conflict resolution specialist
- **Functionality**:
  - Uses Nash equilibrium to balance competing strategies
  - Mathematically optimizes satisfaction vs. cost trade-offs
  - Implements utility functions for competing objectives

### Key Architecture Patterns

#### Strategy Negotiation Flow:
1. Knowledge search → Escalation prediction → Competing strategy proposals → Mediated resolution

#### Technical Highlight:
- **Game Theory Integration**: Uses scipy optimization to find Nash equilibrium between competing objectives
- **Risk-based Resource Allocation**: Matches resource levels (junior/senior/expert) to escalation probability
- **Tier-based SLA Processing**: Different service levels per customer tier

### Strengths
- **Modular Design**: Each agent has clear responsibilities
- **Conflict Resolution**: Formal negotiation mechanism via game theory
- **Risk-aware**: Escalation prediction informs resource allocation
- **Customer-centric**: Differentiates service based on customer value

### Potential Improvements
- Error handling could be more robust (currently basic try-except)
- Knowledge search could benefit from more sophisticated NLP
- Real ML model integration needed (currently uses simple formulas)

In [5]:
# =============================================================================
# CELL 5: MULTI-AGENT SYSTEM ORCHESTRATOR
# =============================================================================

class AdversarialSupportSystem:
    """Main orchestrator for the multi-agent system"""
    
    def __init__(self):
        print("🚀 Initializing Adversarial Multi-Agent Support System...")
        
        generator = SyntheticDataGenerator(num_tickets=500)
        self.tickets_df = generator.generate_tickets()
        self.kb_docs = generator.generate_kb_docs()
        self.sla_rules = generator.generate_sla_rules()
        
        print(f"✅ Generated {len(self.tickets_df)} tickets, {len(self.kb_docs)} KB docs")
        
        self.knowledge_agent = KnowledgeAgent(model, self.kb_docs)
        self.escalation_predictor = EscalationPredictor(model)
        self.advocate_agent = AdvocateAgent(model)
        self.efficiency_agent = EfficiencyAgent(model)
        self.mediator_agent = MediatorAgent(model)
        
        print("✅ All agents initialized\n")
        self.results = []
    
    def process_ticket(self, ticket: Dict) -> Dict:
        """Process a single ticket through the multi-agent system"""
        
        kb_results = self.knowledge_agent.search_solutions(ticket)
        escalation = self.escalation_predictor.predict_escalation(ticket, self.sla_rules)
        
        advocate_strategy = self.advocate_agent.propose_strategy(
            ticket, kb_results, escalation, self.sla_rules
        )
        efficiency_strategy = self.efficiency_agent.propose_strategy(
            ticket, kb_results, escalation, self.sla_rules
        )
        
        final_strategy = self.mediator_agent.negotiate_solution(
            advocate_strategy, efficiency_strategy, ticket, escalation
        )
        
        return {
            "ticket_id": ticket["ticket_id"],
            "kb_solutions": kb_results,
            "escalation_risk": escalation,
            "advocate_proposal": advocate_strategy,
            "efficiency_proposal": efficiency_strategy,
            "final_decision": final_strategy
        }
    
    def process_batch(self, num_tickets: int = 10) -> pd.DataFrame:
        """Process a batch of tickets"""
        print(f"\n{'='*70}")
        print(f"🎯 PROCESSING {num_tickets} TICKETS THROUGH ADVERSARIAL SYSTEM")
        print(f"{'='*70}\n")
        
        sample_tickets = self.tickets_df.head(num_tickets).to_dict('records')
        
        for i, ticket in enumerate(sample_tickets, 1):
            print(f"\n[{i}/{num_tickets}] {ticket['ticket_id']}: {ticket['summary']}")
            print(f"   Tier: {ticket['customer_tier']} | Severity: {ticket['severity']}")
            
            result = self.process_ticket(ticket)
            self.results.append(result)
            
            adv = result['advocate_proposal']
            eff = result['efficiency_proposal']
            final = result['final_decision']
            
            print(f"   💙 Advocate: {adv['response_time_hours']}h, ${adv['estimated_cost']:.0f}, {adv['satisfaction_score']}% sat")
            print(f"   💚 Efficiency: {eff['response_time_hours']}h, ${eff['estimated_cost']:.0f}, {eff['satisfaction_score']}% sat")
            print(f"   ⚖️  Nash: {final['response_time_hours']}h, ${final['estimated_cost']:.0f}, {final['satisfaction_score']:.0f}% sat")
        
        print(f"\n{'='*70}")
        print("✅ BATCH PROCESSING COMPLETE")
        print(f"{'='*70}\n")
        
        return pd.DataFrame(self.results)
    
    def generate_analytics(self) -> Dict:
        """Generate system-wide analytics"""
        if not self.results:
            return {}
        
        analytics = {
            "total_tickets_processed": len(self.results),
            "avg_cost_savings": np.mean([
                r['advocate_proposal']['estimated_cost'] - r['final_decision']['estimated_cost']
                for r in self.results
            ]),
            "avg_satisfaction_score": np.mean([
                r['final_decision']['satisfaction_score'] for r in self.results
            ]),
            "high_risk_tickets": sum([
                1 for r in self.results 
                if r['escalation_risk']['escalation_probability'] > 0.7
            ])
        }
        
        return analytics

print("✅ Multi-agent orchestrator loaded\n")

✅ Multi-agent orchestrator loaded



# Code Summary: Visualization Dashboard

The code below implements an interactive visualization dashboard for analyzing a multi-agent system's performance and negotiation outcomes. Here's a comprehensive summary:

## Overview
The `DashboardVisualizer` class creates four distinct visualizations that provide insights into agent negotiations, cost optimization, risk analysis, and optimal trade-offs in a multi-agent decision system.

## Key Components

### 1. **Negotiation Scatter Plot**
- **Purpose**: Visualizes negotiations between different agents (Advocate, Efficiency, and Nash Equilibrium)
- **Metrics Displayed**: Cost vs Satisfaction, with response time as marker size
- **Visual Features**: Color-coded agents, hover data, interactive exploration

### 2. **Cost Savings Waterfall Chart**
- **Purpose**: Shows cost optimization through the negotiation process
- **Key Metrics**: Total costs from Advocate proposals vs Efficiency proposals vs final Nash equilibrium
- **Features**: Annotations showing savings percentages and optimization results

### 3. **Escalation Risk Heatmap**
- **Purpose**: Analyze escalation risks across different customer tiers and severity levels
- **Dimensions**: Customer tiers (bronze, silver, gold, enterprise) vs severity levels
- **Visualization**: Color-coded risk matrix using RdYlGn_r color scale

### 4. **Pareto Frontier Visualization**
- **Purpose**: Identifies optimal trade-off points between cost and satisfaction
- **Methodology**: Calculates Pareto-optimal solutions from all final decisions
- **Features**: Highlights the efficient frontier among all possible solutions

## Technical Implementation

### Data Structure Requirements
- **`self.results`**: List of dictionaries containing negotiation outcomes for each ticket
- **`self.system.tickets_df`**: DataFrame with customer and ticket information

### Dependencies
- **Plotly** for interactive visualizations
- **Pandas** for data manipulation
- **NumPy** for numerical operations

### Visualization Features
- Dark theme template for modern appearance
- Interactive hover information
- Custom color schemes for consistent branding
- Professional layout with clear axis labels and titles

## User Interface
The `display_all_dashboards()` method provides a complete user experience:
- Sequential rendering of all four visualizations
- Progress indicators and completion messages
- Clean output formatting with visual separators

## Business Value
This dashboard enables stakeholders to:
- Track negotiation effectiveness across different agent strategies
- Quantify cost savings from multi-agent optimization
- Identify risk patterns based on customer characteristics
- Visualize optimal trade-off points for strategic decision-making

In [6]:
# =============================================================================
# CELL 6: VISUALIZATION DASHBOARD
# =============================================================================

class DashboardVisualizer:
    """Create interactive visualizations for the multi-agent system"""
    
    def __init__(self, system, results_df):
        self.system = system
        self.results_df = results_df
        self.results = system.results
        
    def create_negotiation_scatter(self):
        """Scatter plot showing agent negotiations"""
        
        data_points = []
        for r in self.results:
            ticket_id = r['ticket_id']
            
            data_points.append({
                'ticket_id': ticket_id, 'agent': 'Advocate',
                'cost': r['advocate_proposal']['estimated_cost'],
                'satisfaction': r['advocate_proposal']['satisfaction_score'],
                'response_time': r['advocate_proposal']['response_time_hours']
            })
            
            data_points.append({
                'ticket_id': ticket_id, 'agent': 'Efficiency',
                'cost': r['efficiency_proposal']['estimated_cost'],
                'satisfaction': r['efficiency_proposal']['satisfaction_score'],
                'response_time': r['efficiency_proposal']['response_time_hours']
            })
            
            data_points.append({
                'ticket_id': ticket_id, 'agent': 'Nash Equilibrium',
                'cost': r['final_decision']['estimated_cost'],
                'satisfaction': r['final_decision']['satisfaction_score'],
                'response_time': r['final_decision']['response_time_hours']
            })
        
        df = pd.DataFrame(data_points)
        
        fig = px.scatter(df, x='cost', y='satisfaction', color='agent', size='response_time',
                        hover_data=['ticket_id', 'response_time'],
                        title='Agent Negotiation Space: Cost vs Satisfaction',
                        color_discrete_map={'Advocate': '#3b82f6', 'Efficiency': '#10b981', 'Nash Equilibrium': '#a855f7'})
        
        fig.update_layout(xaxis_title='Estimated Cost ($)', yaxis_title='Customer Satisfaction Score (%)',
                         height=600, template='plotly_dark', hovermode='closest')
        
        return fig
    
    def create_cost_savings_waterfall(self):
        """Waterfall chart showing cost optimization"""
        
        total_advocate_cost = sum([r['advocate_proposal']['estimated_cost'] for r in self.results])
        total_efficiency_cost = sum([r['efficiency_proposal']['estimated_cost'] for r in self.results])
        total_nash_cost = sum([r['final_decision']['estimated_cost'] for r in self.results])
        
        savings_from_advocate = total_advocate_cost - total_nash_cost
        savings_from_efficiency = total_nash_cost - total_efficiency_cost
        
        fig = go.Figure(go.Waterfall(
            name="Cost Optimization",
            orientation="v",
            measure=["absolute", "relative", "relative", "total"],
            x=["Advocate<br>Proposal", "Savings from<br>Negotiation", "Additional Cost<br>vs Efficiency", "Final Nash<br>Equilibrium"],
            textposition="outside",
            text=[f"${total_advocate_cost:.0f}", 
                  f"-${savings_from_advocate:.0f}",
                  f"+${savings_from_efficiency:.0f}",
                  f"${total_nash_cost:.0f}"],
            y=[total_advocate_cost, 
               -savings_from_advocate,
               savings_from_efficiency,
               total_nash_cost],
            connector={"line": {"color": "rgb(63, 63, 63)", "width": 2}},
            decreasing={"marker": {"color": "#10b981", "line": {"color": "#059669", "width": 2}}},
            increasing={"marker": {"color": "#ef4444", "line": {"color": "#dc2626", "width": 2}}},
            totals={"marker": {"color": "#a855f7", "line": {"color": "#9333ea", "width": 2}}}
        ))
        
        fig.update_layout(
            title="Cost Optimization Through Nash Equilibrium<br><sub>Balancing Premium Service (Advocate) vs Cost Efficiency</sub>",
            yaxis_title="Total Cost ($)",
            height=600,
            template='plotly_dark',
            showlegend=False,
            xaxis={'type': 'category'}
        )
        
        # Add annotations for context
        fig.add_annotation(
            x="Advocate<br>Proposal",
            y=total_advocate_cost,
            text=f"Starting: ${total_advocate_cost:.0f}",
            showarrow=False,
            yshift=40,
            font=dict(color="#3b82f6", size=10)
        )
        
        fig.add_annotation(
            x="Final Nash<br>Equilibrium",
            y=total_nash_cost,
            text=f"Optimized: ${total_nash_cost:.0f}<br>Savings: ${savings_from_advocate:.0f} ({savings_from_advocate/total_advocate_cost*100:.1f}%)",
            showarrow=False,
            yshift=50,
            font=dict(color="#a855f7", size=10)
        )
        
        return fig
    
    def create_escalation_risk_heatmap(self):
        """Heatmap of escalation risk by tier and severity"""
        
        tickets_df = self.system.tickets_df.head(len(self.results))
        risk_data = []
        
        for idx, r in enumerate(self.results):
            ticket = tickets_df.iloc[idx]
            risk_data.append({
                'tier': ticket['customer_tier'],
                'severity': ticket['severity'],
                'risk': r['escalation_risk']['escalation_probability']
            })
        
        risk_df = pd.DataFrame(risk_data)
        pivot = risk_df.pivot_table(values='risk', index='severity', columns='tier', aggfunc='mean')
        
        tier_order = ['bronze', 'silver', 'gold', 'enterprise']
        pivot = pivot.reindex(columns=[t for t in tier_order if t in pivot.columns])
        
        fig = go.Figure(data=go.Heatmap(
            z=pivot.values, x=[t.title() for t in pivot.columns], y=pivot.index,
            colorscale='RdYlGn_r', text=np.round(pivot.values, 2),
            texttemplate='%{text}', textfont={"size": 12},
            colorbar=dict(title="Risk Score")
        ))
        
        fig.update_layout(title='Escalation Risk Heatmap: Severity vs Customer Tier',
                         xaxis_title='Customer Tier', yaxis_title='Severity Level',
                         height=500, template='plotly_dark')
        
        return fig
    
    def create_pareto_frontier(self):
        """Pareto frontier showing optimal trade-offs"""
        
        data_points = []
        for r in self.results:
            ticket_id = r['ticket_id']
            tier_row = self.system.tickets_df[self.system.tickets_df['ticket_id'] == ticket_id]
            tier = tier_row['customer_tier'].values[0] if len(tier_row) > 0 else 'unknown'
            
            data_points.append({
                'cost': r['final_decision']['estimated_cost'],
                'satisfaction': r['final_decision']['satisfaction_score'],
                'ticket_id': ticket_id,
                'tier': tier
            })
        
        df = pd.DataFrame(data_points)
        
        pareto_points = []
        sorted_df = df.sort_values('cost')
        max_sat = 0
        
        for _, row in sorted_df.iterrows():
            if row['satisfaction'] >= max_sat:
                pareto_points.append(row)
                max_sat = row['satisfaction']
        
        pareto_df = pd.DataFrame(pareto_points)
        
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=df['cost'], y=df['satisfaction'], mode='markers', name='All Solutions',
            marker=dict(size=8, color='#64748b', opacity=0.6), text=df['ticket_id'],
            hovertemplate='<b>%{text}</b><br>Cost: $%{x:.0f}<br>Satisfaction: %{y:.1f}%'
        ))
        
        fig.add_trace(go.Scatter(
            x=pareto_df['cost'], y=pareto_df['satisfaction'], mode='lines+markers', name='Pareto Frontier',
            line=dict(color='#a855f7', width=3), marker=dict(size=12, color='#a855f7'),
            text=pareto_df['ticket_id'],
            hovertemplate='<b>%{text}</b><br>Cost: $%{x:.0f}<br>Satisfaction: %{y:.1f}%'
        ))
        
        fig.update_layout(title='Pareto Frontier: Optimal Cost-Satisfaction Trade-offs',
                         xaxis_title='Cost ($)', yaxis_title='Satisfaction Score (%)',
                         height=600, template='plotly_dark', hovermode='closest')
        
        return fig
    
    def display_all_dashboards(self):
        """Display all visualizations"""
        
        print("\n" + "="*70)
        print("📊 GENERATING INTERACTIVE DASHBOARDS")
        print("="*70 + "\n")
        
        dashboards = [
            ("Negotiation Space", self.create_negotiation_scatter),
            ("Cost Savings Waterfall", self.create_cost_savings_waterfall),
            ("Escalation Risk Heatmap", self.create_escalation_risk_heatmap),
            ("Pareto Frontier", self.create_pareto_frontier)
        ]
        
        for name, func in dashboards:
            print(f"📈 Rendering: {name}")
            fig = func()
            fig.show()
        
        print("\n" + "="*70)
        print("✅ ALL DASHBOARDS RENDERED")
        print("="*70 + "\n")

print("✅ Visualization dashboard loaded\n")

✅ Visualization dashboard loaded



# Code Summary: Run The Complete System

This is the main execution cell that runs the complete system.

**System Overview:**
- An adversarial multi-agent enterprise support system
- Uses game theory-based agent negotiation
- Built with Google Gemini 2.0 Flash
- Processes batches of support tickets
- Generates analytics and visualizations

**Current Implementation Features:**
- Batch processing capability (currently set to 10 tickets, scalable to 500)
- Analytics generation with key metrics
- Dashboard visualization system
- CSV export functionality

In [7]:
# =============================================================================
# CELL 7: RUN THE COMPLETE SYSTEM
# =============================================================================

print("\n" + "="*70)
print("🎮 ADVERSARIAL MULTI-AGENT ENTERPRISE SUPPORT SYSTEM")
print("="*70)
print("📌 Novel Approach: Game Theory-Based Agent Negotiation")
print("📌 Using: Google Gemini 2.0 Flash")
print("="*70 + "\n")

# Initialize system
system = AdversarialSupportSystem()

# Process tickets (start with 10, increase to 500 for full run)
results_df = system.process_batch(num_tickets=10)

# Generate analytics
analytics = system.generate_analytics()

print("\n" + "="*70)
print("📊 SYSTEM ANALYTICS")
print("="*70)
for key, value in analytics.items():
    if isinstance(value, float):
        print(f"   {key}: {value:.2f}")
    else:
        print(f"   {key}: {value}")

# Create visualizations
viz = DashboardVisualizer(system, results_df)
viz.display_all_dashboards()

print("\n" + "="*70)
print("✨ SYSTEM DEMONSTRATION COMPLETE")
print("="*70)
print("\n📌 To process all 500 tickets: system.process_batch(num_tickets=500)")
print("📌 To export results: results_df.to_csv('adversarial_results.csv')")
print("📌 To view specific dashboard: viz.create_negotiation_scatter().show()")


🎮 ADVERSARIAL MULTI-AGENT ENTERPRISE SUPPORT SYSTEM
📌 Novel Approach: Game Theory-Based Agent Negotiation
📌 Using: Google Gemini 2.0 Flash

🚀 Initializing Adversarial Multi-Agent Support System...
✅ Generated 500 tickets, 24 KB docs
✅ All agents initialized


🎯 PROCESSING 10 TICKETS THROUGH ADVERSARIAL SYSTEM


[1/10] TKT-0001: Analytics - slow queries
   Tier: bronze | Severity: 3
   💙 Advocate: 16h, $1200, 80% sat
   💚 Efficiency: 43h, $700, 60% sat
   ⚖️  Nash: 29h, $350, 100% sat

[2/10] TKT-0002: Security - compliance questions
   Tier: bronze | Severity: 5
   💙 Advocate: 16h, $1200, 80% sat
   💚 Efficiency: 43h, $700, 60% sat
   ⚖️  Nash: 29h, $350, 100% sat

[3/10] TKT-0003: Security - compliance questions
   Tier: bronze | Severity: 2
   💙 Advocate: 16h, $1200, 80% sat
   💚 Efficiency: 43h, $700, 60% sat
   ⚖️  Nash: 29h, $350, 100% sat

[4/10] TKT-0004: CRM - contact import errors
   Tier: bronze | Severity: 4
   💙 Advocate: 16h, $1200, 80% sat
   💚 Efficiency: 43h, $700, 60%

📈 Rendering: Cost Savings Waterfall


📈 Rendering: Escalation Risk Heatmap


📈 Rendering: Pareto Frontier



✅ ALL DASHBOARDS RENDERED


✨ SYSTEM DEMONSTRATION COMPLETE

📌 To process all 500 tickets: system.process_batch(num_tickets=500)
📌 To export results: results_df.to_csv('adversarial_results.csv')
📌 To view specific dashboard: viz.create_negotiation_scatter().show()


## Code Summary: Optional CSV Data Export

This code segment provides **optional data export functionality** for saving analysis results to CSV files. The code is designed to export three key datasets:

### IMPORTANT NOTE
As mentioned earlier, the default number of datasets to be processed is set to **10** to optimize the compute resources. If you would like to process all 500 records, follow these steps:

**1. Open the command line Console. It can be found at the bottom left-hand side of the screen; it looks like a Command Shell icon.**
**2. Copy/Paste the desired export command from below and hit ENTER.
**3. The output will be saved in Kaggle's Output Space, which can be turned on by going to the View / Show Sidebar menu item and expanding the /kaggle/working folder.**

### Core Functionality
- **Saves results:** `results_df.to_csv('results.csv', index=False)`
- **Exports tickets:** `system.tickets_df.to_csv('tickets.csv', index=False)`
- **Exports knowledge base documents:** `system.kb_docs.to_csv('kb_docs.csv', index=False)`

### Key Characteristics

1. **Optional Implementation:** The code is explicitly marked as optional and must be uncommented to activate
2. **Standard CSV Export:** Uses pandas DataFrame's `to_csv()` method with `index=False` parameter
3. **Clean Output:** The `index=False` parameter ensures exported files don't include pandas indices, keeping the data clean

### Technical Details

- **File Format:** Standard CSV (Comma-Separated Values) format
- **Index Handling:** `index=False` parameter excludes row indices from output files
- **Default Behavior:** Files will be saved in the current working directory unless full paths are specified

### Purpose
This export functionality is typically used for:
- Persisting analysis results for future reference
- Sharing processed data with stakeholders
- Creating backup copies of processed data
- Enabling data import into other systems or tools

### Best Practices Used
- Files are logically named for easy identification
- Consistent parameter usage across all exports
- Clean data export without unnecessary index columns

In [8]:
# =============================================================================
# OPTIONAL: EXPORT DATA
# =============================================================================

# Uncomment to save results
results_df.to_csv('results.csv', index=False)
system.tickets_df.to_csv('tickets.csv', index=False)
system.kb_docs.to_csv('kb_docs.csv', index=False)